In [468]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from catboost import CatBoostRegressor

In [469]:
def error(y_train,y_pred):
    print(mean_absolute_error(y_train,y_pred))
    print(mean_absolute_percentage_error(y_train,y_pred))

In [558]:
df = pd.read_csv('C:\\Users\\zizzs\\Desktop\\Учица\\Train\\realtor-data.csv')

In [547]:
df

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,NaN,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,NaN,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,NaN,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,NaN,145000.0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,NaN,NaN,65000.0
...,...,...,...,...,...,...,...,...,...,...
99995,for_sale,3.0,3.0,NaN,Hudson,Massachusetts,1749.0,2864.0,NaN,749900.0
99996,for_sale,2.0,1.0,0.34,Auburn,Massachusetts,1501.0,1075.0,1999-06-07,349900.0
99997,for_sale,3.0,2.0,1.01,Shrewsbury,Massachusetts,1545.0,1632.0,1995-09-27,549000.0
99998,for_sale,3.0,2.0,0.12,Worcester,Massachusetts,1604.0,1332.0,2000-09-11,299000.0


In [559]:
df[df['bed'].isna()] = df['bed'].median()               #Clearing the Dataset from Nan's
df[df['bath'].isna()] = df['bath'].median()             #
df[df['acre_lot'].isna()] = df['acre_lot'].median()     #
df[df['house_size'].isna()] = df['house_size'].median() #
df = df.drop('zip_code',axis=1).copy()                  #Dropping zip_code, because it is not worth counting
df = df.drop('prev_sold_date',axis=1).copy()            #Dropping prev_sold_date because it is hard to get from all Nan's in this column
df = df.loc[df['price'] > 15000.0].copy()               #There are many strange filled Series(like status = 0.17 or price = 0.17), and all of them are getting away with this part
df = df.dropna(subset=['city']).copy()                  #Dropping one last Nan from city column

train, test = train_test_split(df,train_size=0.6,random_state=137) #Splitting dataset for Catboost on Train(60%), Test(20%), Val(20%)
test, val = train_test_split(test,train_size=0.5,random_state=137)
df.dtypes                                               #showing types of columns

status         object
bed           float64
bath          float64
acre_lot      float64
city           object
state          object
house_size    float64
price         float64
dtype: object

In [467]:
# dfcopy = df.copy()

In [557]:
train

,status,bed,bath,acre_lot,city,state,house_size,price
68198,for_sale,7.0,2.0,0.13,Springfield,Massachusetts,2481.0,269900.0
94733,for_sale,3.0,1.0,0.08,West Warwick,Rhode Island,1142.0,279000.0
47101,for_sale,6.0,2.0,1.26,Manchester,Connecticut,1808.0,325000.0
87638,for_sale,8.0,5.0,0.23,Spencer,Massachusetts,4219.0,355000.0
43533,for_sale,3.0,1.0,0.73,Palmer,Massachusetts,1250.0,269900.0
...,...,...,...,...,...,...,...,...
91281,for_sale,2.0,1.0,0.88,Westborough,Massachusetts,815.0,325000.0
1863,for_sale,4.0,1.0,0.94,Utuado,Puerto Rico,13818.0,108000.0
90300,for_sale,3.0,2.0,0.39,Leominster,Massachusetts,2205.0,436590.0
86903,for_sale,3.0,1.0,0.15,Marlborough,Massachusetts,1617.0,344900.0


In [560]:
df.isna().mean()

status        0.0
bed           0.0
bath          0.0
acre_lot      0.0
city          0.0
state         0.0
house_size    0.0
price         0.0
dtype: float64

In [474]:
train.columns

Index(['status', 'bed', 'bath', 'acre_lot', 'city', 'state', 'house_size',
       'price'],
      dtype='object')

In [530]:
X = ['bed', 'bath', 'acre_lot', 'city', 'state', 'house_size']

cat_features = ['city', 'state']

y = ['price']

In [533]:
model = CatBoostRegressor(cat_features=cat_features,
                        learning_rate=1,
                        eval_metric='MAPE',
                        iterations=10000,
                        random_seed=137,
                        verbose=100)


model = CatBoostRegressor(learning_rate=1,
                        eval_metric='MAPE',
                        iterations=10000,
                        random_seed=137,
                        verbose=100)

In [534]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 1.0171704	test: 1.0176564	best: 1.0176564 (0)	total: 4.96ms	remaining: 49.6s
100:	learn: 0.4081552	test: 0.4308355	best: 0.4308355 (100)	total: 690ms	remaining: 1m 7s
200:	learn: 0.3375973	test: 0.3651872	best: 0.3651872 (200)	total: 1.27s	remaining: 1m 2s
300:	learn: 0.2806776	test: 0.3137961	best: 0.3137961 (300)	total: 1.91s	remaining: 1m 1s
400:	learn: 0.2490355	test: 0.2858065	best: 0.2858065 (400)	total: 2.51s	remaining: 1m
500:	learn: 0.2185890	test: 0.2578191	best: 0.2578191 (500)	total: 3.09s	remaining: 58.7s
600:	learn: 0.1962012	test: 0.2391775	best: 0.2391775 (600)	total: 3.77s	remaining: 58.9s
700:	learn: 0.1756726	test: 0.2178847	best: 0.2178847 (700)	total: 4.59s	remaining: 1m
800:	learn: 0.1597634	test: 0.2013186	best: 0.2012299 (799)	total: 6.58s	remaining: 1m 15s
900:	learn: 0.1469766	test: 0.1882492	best: 0.1882492 (900)	total: 7.46s	remaining: 1m 15s
1000:	learn: 0.1362792	test: 0.1784105	best: 0.1784105 (1000)	total: 8.35s	remaining: 1m 15s
1100:	learn: 0

In [563]:
test['testprediction'] = model.predict(test[X])

In [564]:
error(test['price'],test['testprediction'])

14623.320421204166
0.06440894539454266
